**Programador preterminado dask**
    
**Threads vs. processes**

*Thredas (Hilos) :*

    - Dask arrays
    - Dask dataframes
    - Delayed pipelines creados con dask.delayed()
    - Son muy rápidos para inicia
    - No es necesario transferirles datos
    - Están limitados por el GIL, lo que permite que un subproceso lea el código de una sola vez
    
*Processes (Procesos) :*
    
    - Dask bags
    - Tómese el tiempo para configurar
    - Lento para transferir datos
    - Cada uno tiene su propio GIL, por lo que no es necesario turnarse para leer el código

*Elegir el programador*

In [1]:
import dask.dataframe as dd
import dask

df  = dd.read_csv("spotify/*.csv", blocksize="1MB")

In [2]:
x = df["tempo"].round()

#Uso normal
result = x.compute()
result = dask.compute(x)
result

(0       114.0
 1       112.0
 2       121.0
 3       170.0
 4       160.0
         ...  
 5378    120.0
 5379    140.0
 5380    127.0
 5381     90.0
 5382     92.0
 Name: tempo, Length: 161738, dtype: float64,)

In [3]:
#Uso de threads (hilos)
result = x.compute(scheduler = 'threads')
result = dask.compute(x, sheduler = 'threads')
result

(0       114.0
 1       112.0
 2       121.0
 3       170.0
 4       160.0
         ...  
 5378    120.0
 5379    140.0
 5380    127.0
 5381     90.0
 5382     92.0
 Name: tempo, Length: 161738, dtype: float64,)

In [4]:
# Use processes
result = x.compute(scheduler='processes')
result = dask.compute(x, scheduler='processes')
result

(0       114.0
 1       112.0
 2       121.0
 3       170.0
 4       160.0
         ...  
 5378    120.0
 5379    140.0
 5380    127.0
 5381     90.0
 5382     92.0
 Name: tempo, Length: 161738, dtype: float64,)

**Clústeres y clientes**

Según el hardware de su computadora y el cálculo que está tratando de completar, puede ser más rápido ejecutarlo usando una combinación de subprocesos y procesos.

Para ello, debe configurar un clúster local.

Hay dos formas de configurar un clúster local que usará Dask.

La primera forma es crear el clúster local y pasarlo a un cliente.

¡Esto es muy similar a cómo configuraría un cliente para que se ejecute en un grupo de computadoras! 

La segunda forma es usar el cliente directamente y permitirle crear el clúster local por sí mismo.

Este es un atajo que funciona para clústeres locales, pero no para otros tipos de clústeres.

Creamos clientes utilizando ambos métodos.

Tenga cuidado al crear el clúster y los clientes. Si los configura incorrectamente, su sesión puede expirar.

**Creación de un cluster local**

In [5]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    processes=True,
    n_workers=2,
    threads_per_worker=2)

print(cluster)

LocalCluster(63d35828, 'tcp://127.0.0.1:51446', workers=2, threads=4, memory=39.69 GiB)


In [ ]:
cluster = LocalCluster(
    processes=False,
    n_workers=2,
    threads_per_worker=2)

print(cluster)

**Un cluster simple**

In [ ]:
cluster = LocalCluster(processes=True)

print(cluster)

**Creación de un cliente**

In [6]:
client = Client(cluster)

print(client)

<Client: 'tcp://127.0.0.1:51446' processes=2 threads=4, memory=39.69 GiB>


Uso del cluster

In [9]:
result = x.compute()
result = x.compute(scheduler='threads')
result = client.compute(x)

In [10]:
result

<Future: finished, type: pandas.core.series.Series, key: finalize-cf31f672400f671f54038bf826501b96>

### Entrenamiento de modelos machine learning con grandes conjuntos de datos

Usando Dask para entrenar un modelo lineal

Dask se puede usar para entrenar modelos de aprendizaje automático en conjuntos de datos que son demasiado grandes para caber en la memoria y le permite distribuir la carga de datos, el preprocesamiento y el entrenamiento en varios subprocesos, procesos e incluso en varias computadoras.

Se le asignó la tarea de entrenar un modelo de aprendizaje automático que predecirá la popularidad de las canciones en el conjunto de datos de Spotify.

Los datos se cargaran como Dask DataFrames perezosos.

Las variables de entrada están disponibles dask_X y contienen algunas columnas numéricas, como el tempo y la capacidad de baile de la canción.

Los valores objetivo están disponibles como dask_y y son la puntuación de popularidad de cada canción.

In [ ]:
from sklearn.linear_model import SGDRegressor
from dask_ml.wrappers import Incremental

# Create a SGDRegressor model
model = SGDRegressor()

# Wrap the model so that it works with Dask
dask_model = Incremental(model, scoring='neg_mean_squared_error')

# Fit the wrapped model
dask_model.fit(dask_X, dask_y)

Cada vez que ejecuta el .fit() método, Dask optimiza el cálculo copiando el modelo en el proceso o subproceso donde se encuentran los datos, en lugar de copiar los datos en el proceso principal que contiene el modelo.

Puede llevar mucho tiempo copiar la información y el modelo es mucho más pequeño que el conjunto de datos, por lo que es mucho más eficiente.

**Haciendo predicciones perezosas**

El modelo que entrenó la última vez fue bueno, pero podría ser mejor si pasara los datos de entrenamiento unas cuantas veces más.

Además, es una pena ver que un buen modelo se desperdicia, por lo que debe usar este para hacer algunas predicciones en un conjunto de datos separado del que entrena.


In [ ]:
# Loop over the training data 5 times
for i in range(5):

    dask_model.partial_fit(dask_X, dask_y)

# Use your model to make predictions
y_pred_delayed = dask_model.predict(dask_X)

# Compute the predictions
y_pred_computed = y_pred_delayed.compute()

print(y_pred_computed)

¡Fantástico! Ese es un modelo bien ajustado.

Si solo usara el .fit() método 5 veces, su código se ejecutaría, pero no obtendría predicciones más precisas en cada repetición del ciclo.

Cuando .fit() se ejecuta, el modelo se restablece a un estado no ajustado y se vuelve a ajustar a los datos, por lo que comienza desde cero cada vez.

El uso del .partial_fit() método nos permite retomar el ajuste desde donde lo dejamos y refinar el ajuste del bucle anterior.

### Reprocesando grandes conjuntos de datos

**Transformación perezosa de datos de entrenamiento**

El preprocesamiento de las variables de entrada es un paso vital en el aprendizaje automático y, a menudo, mejorará la precisión del modelo que cree.

En el último par de ejercicios, los datos de Spotify fueron preprocesados, pero es importante que sepa cómo hacerlo usted mismo.

En este ejercicio, utilizará el StandardScaler() objeto escalador, que transforma las columnas de una matriz para que tengan una media de cero y una desviación estándar de uno.


In [ ]:
# Import the StandardScaler class
from dask_ml.preprocessing import StandardScaler

X = dask_df[['duration_ms', 'explicit', 'danceability', 'acousticness', 'instrumentalness', 'tempo']]

# Select the target variable
y = dask_df['popularity']

# Create a StandardScaler object and fit it on X
scaler = StandardScaler()
scaler.fit(X)

# Transform X
X = scaler.transform(X)
print(X)

¡Bien hecho! 

Es posible que haya notado que X sigue siendo un Dask DataFrame incluso después de haber sido transformado.

Sin embargo, ya ha tenido que cargar todos los datos X una vez para poder ajustar el archivo scaler.

**División de prueba de tren perezoso**

Has transformado las X variables.

Ahora necesita terminar su preparación de datos transformando las y variables y dividiendo sus datos en conjuntos de entrenamiento y prueba.

In [ ]:
# Import the train_test_split function
from dask_ml.model_selection import train_test_split

# Rescale the target values
y = y / 100

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2)

print(X_train)